# ILAB-03: Trực quan hoá dữ liệu với t-SNE

**Họ tên**: Tôn Anh Huy

**MSSV**: 21120257

Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

/Users/huyton/opt/anaconda3/envs/min_ds-env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Giới thiệu về dữ liệu

`glove.6B.50d` là một tập dữ liệu chứa các vector nhúng từ (word embedding) được đào tạo trên một tập dữ liệu khổng lồ gồm 6 tỷ từ. Mỗi từ trong tập dữ liệu được biểu diễn bằng một vector gồm 50 số thực, đại diện cho các đặc điểm ngữ nghĩa của từ đó.

Link của dataset gốc: https://nlp.stanford.edu/projects/glove/

Do mục đích minh hoạ cho t-SNE, dữ liệu đã được giảm bớt kích cỡ bằng cách lọc theo một vài topic nhất định

Bộ dữ liệu này tương đối nhẹ và dễ dàng thực hiện. Bởi vì nếu sử dụng `t-sne` thì có độ phức tạp rất cao $O(n^2)$, nếu dùng dataset quá lớn thì thời gian để fit rất lâu

In [2]:
df = pd.read_csv('sub_glove.6B.50d.csv', header=None)
df.rename(columns={0: 'word', 51: 'topic'}, inplace=True)
df.head()

,word,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,topic
0,fish,0.25590,-0.314520,-1.007900,0.143400,0.52982,0.31531,-0.82169,0.006003,1.52350,...,0.21292,1.07230,0.32437,1.33040,0.54586,-0.222360,-0.805890,0.164870,0.29792,animals
1,bird,0.78675,0.079368,-0.765970,0.193100,0.55014,0.26493,-0.75841,-0.881800,1.64680,...,0.31817,0.29354,0.25109,1.61110,0.71340,-0.152430,-0.253620,0.264190,0.15875,animals
2,dog,0.11008,-0.387810,-0.576150,-0.277140,0.70521,0.53994,-1.07860,-0.401460,1.15040,...,0.24068,-0.64819,0.83549,1.25020,-0.51379,0.042240,-0.881180,0.715800,0.38519,animals
3,tiger,0.33183,-0.602560,-0.046032,0.899410,0.26504,-0.02695,-0.89378,-0.219960,0.28162,...,-0.46533,-0.30380,0.11061,1.32160,-0.34734,0.039552,0.046421,-0.086983,-0.45584,animals
4,cat,0.45281,-0.501080,-0.537140,-0.015697,0.22191,0.54602,-0.67301,-0.689100,0.63493,...,0.48138,-0.43352,0.69455,0.91043,-0.28173,0.416370,-1.260900,0.712780,0.23782,animals


# Giảm chiều dữ liệu với PCA

Bây giờ, ta thử thực hiện giảm chiều của dữ liệu gốc về 2 chiều.

Đồng thời, so sánh luôn tốc độ fit với dữ liệu của PCA so với t-SNE

In [3]:
X = df.drop(['word', 'topic'], axis=1)

In [4]:
%%time
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

CPU times: user 10.3 ms, sys: 13.7 ms, total: 24.1 ms
Wall time: 22.1 ms


Thời gian fit với dữ liệu của PCA rất nhanh

In [5]:
fig = px.scatter(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    color=df['topic'],
    text=df['word'],
    title='PCA',
    labels={'x': '1st dimension', 'y': '2st dimension'},
    # change legend title
)

fig.update_layout(legend_title_text='Topic')

fig.update_traces(
    textposition='top center',
    marker=dict(size=12,
                line=dict(width=2,
                          color='DarkSlateGrey'))
)

**Nhận xét**

- Dễ dàng thấy giữa các clusters có sự trùng lắp với nhau

- PCA thể hiện tốt trong việc bảo toàn cấu trúc tổng thể của dữ liệu, giữ lại được nhiều thông tin nhất có thể. Tuy nhiên không thể hiện rõ nét mối quan hệ giữa các cụm dữ liệu.

- Việc sử dụng tương đối dễ dàng do không yêu cầu chỉ định các siêu tham số

# Giảm chiều dữ liệu với t-SNE

Ý tưởng chính của t-SNE là biểu diễn các dữ liệu ở không gian có số chiều lớn về không gian có số chiều thấp, thông thường là 2-3 chiều, nhưng vẫn duy trì được mối quan hệ cục bộ giữa các điểm dữ liệu. Thuật toán xây dựng một phân phối đại diện cho sự tương tự của các điểm dữ liệu trong không gian gốc và phân phối tương tự trong không gian có số chiều thấp. Thuật toán sẽ cố gắng tối tiểu hoá sự sai khác giữa hai phân phối này

Bây giờ ta sẽ đưa dữ liệu gốc ở không gian có số chiều cao (50), về không gian có số chiều thấp (2).

Đồng thời, đo thời gian để khớp với dữ liệu của `t-SNE` so với `PCA`

In [6]:
%%time
tsne = TSNE(n_components=2, random_state=0)
X_transformed = tsne.fit_transform(X)

CPU times: user 1.48 s, sys: 1.1 s, total: 2.58 s
Wall time: 498 ms


Thời gian fit với dữ liệu của t-sne tương đối lâu so với pca. Điều này dễ hiểu vì t-sne thường yêu cầu số lần lặp đủ lớn để có thể đạt được trạng thái phù hợp, việc này có thể còn lâu hơn nếu người dùng đưa ra số vòng lặp lớn hơn.

In [7]:
fig = px.scatter(
    x=X_transformed[:, 0],
    y=X_transformed[:, 1],
    color=df['topic'],
    text=df['word'],
    title='t-SNE',
    labels={'x': '1st dimension', 'y': '2st dimension'},
)

fig.update_layout(legend_title_text='Topic')

fig.update_traces(
    textposition='top center',
    marker=dict(size=12,
                line=dict(width=2,
                          color='DarkSlateGrey'))
)

**Nhận xét**

- Các clusters được phân tách rõ ràng, tình trạng overlap giữa các cluster đã được giảm đi đáng kể.

- Có thể thấy rằng, các từ thuộc về một chủ đề nhất định thường nằm trong cùng một cluster. Bởi vì mục tiêu của t-sne là cố gắng duy trì độ tương tự của các điểm dữ liệu trong không gian mới so với ban đầu.

- Ví dụ: các từ thuộc chủ đề "chữ số" như : one, two, three... có xu hướng nằm gần nhau.

# Kết luận

## PCA

### Điểm mạnh

- Hiệu quả trong tính toán: PCA có hiệu quả tính toán, đặc biệt là đối với các bộ dữ liệu lớn, làm cho nó phù hợp cho dữ liệu có số chiều cao.

- Bảo toàn cấu trúc toàn cục của dữ liệu: PCA tập trung vào việc bảo toàn cấu trúc toàn cục của dữ liệu bằng cách giữ lại phương sai lớn nhất.

- Dễ dàng truyền tải: các thành phần chính từ PCA là tổ hợp tuyến tính của features ban đầu, làm cho chúng có thể được giải thích dễ dàng theo các đặc điểm gốc.

- Trích lọc đặc trưng: giúp đơn giản hoá dữ liệu ban đầu, giúp quá trình huấn luyện mô hình học máy trở nên hiệu quả hơn.


### Điểm yếu

- Giả định về sự tuyến tính: PCA giả định rằng cấu trúc của dữ liệu là tuyến tính, điều này có thể không đúng với các bộ dữ liệu phức tạp với mối quan hệ phi tuyến.

- Mất cấu trúc địa phương: PCA có thể không bảo toàn cấu trúc cục bộ hoặc mối quan hệ giữa các điểm dữ liệu gần nhau

- Nhạy cảm với ngoại lai: việc tối đa hóa phương sai có thể bị ảnh hưởng nặng nề bởi các giá trị ngoại lai

### Các trường hợp sử dụng

- Mục tiêu giảm chiều dữ liệu

- Ưu tiên hiệu suất tính toán

- Lựa chọn đặc trưng cho học máy


## t-SNE

### Điểm mạnh

- Giảm chiều phi tuyến tính: t-SNE hiệu quả trong việc phát hiện các mối quan hệ phi tuyến tính giữa các điểm dữ liệu

- Bảo toàn cấu trúc cục bộ: t-SNE bảo toàn cấu trúc địa phương của dữ liệu bằng cách mô hình hóa sự tương tự giữa các điểm gần nhau trong không gian có số chiều cao.

- Khả năng trực quan: t-SNE được sử dụng rộng rãi cho mục đích trực quan các cụm và mẫu trong dữ liệu có số chiều cao.

### Điểm yếu

- Chi phí tính toán cao: t-SNE có thể tốn kém về mặt tính toán, đặc biệt là đối với các bộ dữ liệu lớn, và có thể yêu cầu điều chỉnh cẩn thận của các siêu tham số.

- Khó truyền tải: việc giải thích kết quả phức tạp, vì các khoảng cách trong không gian có số chiều thấp không tương ứng trực tiếp với không gian gốc.

### Các trường hợp sử dụng

- Trực quan hoá dữ liệu

- Mạnh mẽ trong trực quan hóa dữ liệu và khám phá các cấu trúc phức tạp



